In [2]:
import json
import os
from collections import Counter
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

###  5-6 Ball_per_over Statistics

In [3]:
ball_per_over=[]

folder_path = "../data/raw/cricsheet/all_json/"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path,'r') as file:
        data=json.load(file)
        ball_per_over.append(data["info"]["balls_per_over"])


counter = Counter(ball_per_over)
print(counter)


Counter({6: 2888})


In [4]:
field_venue=[]

folder_path = "../data/raw/cricsheet/all_json/"

venue_matches={}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path,'r') as file:
        data=json.load(file)
        match_id=filename
        # print(match_id)
        # print(file)
        if "venue" in data["info"]:
            field_venue.append(data["info"]["venue"])
            # print(field_venue)
            if(data["info"]["venue"] in venue_matches):
                venue_matches[data["info"]["venue"]].append(filename)
            else:
                venue_matches[data["info"]["venue"]]=[filename]
            # break


In [5]:
print(f'Unique stadiums :{len(venue_matches.keys())}')

Unique stadiums :367


In [6]:
# storing all the stadiums
with open("pitch_name.txt", "w") as file:
    for pitch in field_venue:
        file.write(pitch+"\n")

### Player Data Statistics

In [7]:
folder_path = "../data/raw/additional_data/player_data.csv"
player_data_df=pd.read_csv(folder_path)

In [8]:
player_data_df.head()

,identifier,full_name,batting_style,bowling_style,playing_role
0,bc005f0d,Aneka Akeilia Aresha White,Right hand Bat,Right arm Medium,NaN
1,5c67c7a6,Azam Ali Baig,NaN,NaN,NaN
2,6a58e069,Arun Bamal,Left hand Bat,Slow Left arm Orthodox,Bowling Allrounder
3,b4a23876,Alex Adrian Anthony Amsterdam,Left hand Bat,Right arm Offbreak,Top order Batter
4,482762af,Adewale A Adeoye,Right hand Bat,Right arm Medium,NaN


In [7]:
player_data_df["batting_style"].unique()

array(['Right hand Bat', nan, 'Left hand Bat'], dtype=object)

In [10]:
player_data_df["bowling_style"].nunique()

56

In [8]:
import requests
from bs4 import BeautifulSoup

# Define the target URL


In [48]:
final_data=[]
name=list(set(field_venue))
for ele in name:
    try:
        url = "https://pitch-report.com/?s="+ele  # Replace with the actual URL
    # temp=[]
    # Send an HTTP request to fetch the page content
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            links = [a["href"] for a in soup.find_all("a", href=True) if ele in a.get_text()]
            response = requests.get(links[0])

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                keywords=["Boundary","Field","Surface"]
                bounday_data=[li.get_text() for li in  soup.find_all("li") if any(keyword in li.getText() for keyword in keywords)]

                divs = soup.find_all("h3")
                div=[div for div in divs if "Batting Pitch Or Bowling Pitch" in div.get_text(strip=True)]

                d=div[0].find_parent("div").find_parent("div").find_parent("div").find_parent("div")
                d=d.find_next_sibling()
                data=[p.get_text() for p in d.find_all('p')]
                bounday_data.append(data[0])
                temp_data=" ".join(data)
                bounday_data.append(temp_data)
                bounday_data.append(ele)
                final_data.append(bounday_data)
    except:
        pass


file_path = "output.csv"
try:
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the rows of the matrix into the CSV
        writer.writerows(final_data)
        
    print(f"Matrix successfully written to {file_path}")
    
except Exception as e:
    print(f"An error occurred: {e}")


Matrix successfully written to output.csv


# extracting the type of stadium and other info about the stadium

In [77]:
final_data=[]
name=list(set(field_venue))
for ele in name:
    try:
        url = "https://pitch-report.com/?s="+ele  # Replace with the actual URL
    # temp=[]
    # Send an HTTP request to fetch the page content
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            links = [a["href"] for a in soup.find_all("a", href=True) if ele in a.get_text()]
            response = requests.get(links[0])

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                keywords=["Boundary","Field","Surface"]
                bounday_data=[li.get_text() for li in  soup.find_all("li") if any(keyword in li.getText() for keyword in keywords)]

                divs = soup.find_all("h3")
                div=[div for div in divs if "Batting Pitch Or Bowling Pitch" in div.get_text(strip=True)]

                d=div[0].find_parent("div").find_parent("div").find_parent("div").find_parent("div")
                d=d.find_next_sibling()
                data=[p.get_text() for p in d.find_all('p')]
                bounday_data.append(data[0])
              
                temp_data=" ".join(data)
                bounday_data.append(temp_data)
                bounday_data.append(ele)
                final_data.append(bounday_data)
                # print(final_data)
                # print(bounday_data)
    except:
        pass
    break




['Field size:\xa0', 'Surface:\xa0', 'Boundary Length (Dimension):\xa0', 'Leg Side Boundary Length:\xa0', 'Off Side Boundary Length:', 'Straight Boundary Length:', 'Back Side Boundary Length:', 'Batting or Bowling Pitch']


In [13]:
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 33.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 16.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 24.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [16]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.1 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
import re
import spacy
import json

# Load the spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

# Input data
# data = [
#     'Field size: 175 mtr x 135 mtr',
#     'Surface: Grass',
#     'Boundary Length (Dimension): 191 Yd x 147 Yd',
#     'Leg Side Boundary Length: 63 meters',
#     'Off Side Boundary Length: 72 meters',
#     'Straight Boundary Length: 96 meters',
#     'Back Side Boundary Length: 79 meters',
#     'Bellerive Oval Cricket Ground is a Batting Pitch.',
#     'Bellerive Oval Cricket Ground is a Batting Pitch. Pitches typically start off slow and gradually flatten out as the match progresses. Over time, the pitch characteristics have evolved, leading to a more balanced contest between bat and ball. While bowlers may enjoy some initial assistance, particularly early on, the conditions tend to favor the batsmen as the game moves forward, making for an exciting and competitive encounter.',
#     'Hobart'
# ]

def processStadium(data):
    # Regex patterns to extract relevant data
    patterns = {
        'field_size': r"Field size:\s*(\d+\s*mtr\s*x\s*\d+\s*mtr)",
        'surface': r"Surface:\s*(\w+)",
        'boundary_length': r"Boundary Length \(Dimension\):\s*(\d+\s*Yd\s*x\s*\d+\s*Yd)",
        'leg_side_boundary': r"Leg Side Boundary Length:\s*(\d+\s*meters)",
        'off_side_boundary': r"Off Side Boundary Length:\s*(\d+\s*meters)",
        'straight_boundary': r"Straight Boundary Length:\s*(\d+\s*meters)",
        'back_side_boundary': r"Back Side Boundary Length:\s*(\d+\s*meters)",
        'pitch_type': r"Cricket Ground is a (Batting Pitch|Bowling Pitch)",
        'location': r"(\w+)$"
    }
    

    # Initialize a dictionary to store the extracted values
    extracted_data = {}
    data_pitch_info=[]
    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, ' '.join(data))
        curr_match=(match.group(1))
        data_pitch_info.append(curr_match)
        if match:
            extracted_data[key] = match.group(1)

    # For pitch description, you can analyze the text further if needed
    pitch_description = [entry for entry in data if "Pitch" in entry]
    if pitch_description:
        extracted_data['pitch_description'] = ' '.join(pitch_description)

    # # Output the result in JSON format
    # json_result = json.dumps(extracted_data, indent=4)
    # print(data_pitch_info)
    return data_pitch_info

final_data=[]
# name=list(set(field_venue))
stopper=0
# type 1 error-> the way of handling web scrapping is different
info_type1_error=[]
for ele in venue_matches:
    stopper+=1
    # if(stopper==5): break
    # try:
    #     url = "https://pitch-report.com/?s="+ele  # Replace with the actual URL
    # # temp=[]
    # # Send an HTTP request to fetch the page content
    #     response = requests.get(url)

    #     if response.status_code == 200:
    #         soup = BeautifulSoup(response.text, "html.parser")
    #         links = [a["href"] for a in soup.find_all("a", href=True) if ele in a.get_text()]
    #         response = requests.get(links[0])

    #         if response.status_code == 200:
    #             soup = BeautifulSoup(response.text, "html.parser")
    #             keywords=["Boundary","Field","Surface"]
    #             bounday_data=[li.get_text() for li in  soup.find_all("li") if any(keyword in li.getText() for keyword in keywords)]

    #             divs = soup.find_all("h3")
    #             div=[div for div in divs if "Batting Pitch Or Bowling Pitch" in div.get_text(strip=True)]

    #             d=div[0].find_parent("div").find_parent("div").find_parent("div").find_parent("div")
    #             d=d.find_next_sibling()
    #             data=[p.get_text() for p in d.find_all('p')]
    #             bounday_data.append(data[0])
    #             temp_data=" ".join(data)
    #             bounday_data.append(temp_data)
    #             bounday_data.append(ele)
    #             final_data.append(bounday_data)
    #             # print(final_data)
    #             # print(bounday_data)
    #             patterns = {
    #                 'field_size': r"Field size:\s*(\d+\s*mtr\s*x\s*\d+\s*mtr)",
    #                 'surface': r"Surface:\s*(\w+)",
    #                 'boundary_length': r"Boundary Length \(Dimension\):\s*(\d+\s*Yd\s*x\s*\d+\s*Yd)",
    #                 'leg_side_boundary': r"Leg Side Boundary Length:\s*(\d+\s*meters)",
    #                 'off_side_boundary': r"Off Side Boundary Length:\s*(\d+\s*meters)",
    #                 'straight_boundary': r"Straight Boundary Length:\s*(\d+\s*meters)",
    #                 'back_side_boundary': r"Back Side Boundary Length:\s*(\d+\s*meters)",
    #                 'pitch_type': r"Cricket Ground is a (Batting Pitch|Bowling Pitch)",
    #                 'location': r"(\w+)$"
    #             }
                

    #             # Initialize a dictionary to store the extracted values
    #             extracted_data = {}
    #             data_pitch_info=[]
    #             # Extract values using regex
    #             for key, pattern in patterns.items():
    #                 match = re.search(pattern, ' '.join(data))
    #                 curr_match=(match.group(1))
    #                 data_pitch_info.append(curr_match)
    #                 if match:
    #                     extracted_data[key] = match.group(1)

    #             # For pitch description, you can analyze the text further if needed
    #             pitch_description = [entry for entry in data if "Pitch" in entry]
    #             if pitch_description:
    #                 extracted_data['pitch_description'] = ' '.join(pitch_description)

    #             print(data_pitch_info)
    #             # break
    # except:
        
    url = "https://pitch-report.com/?s="+ele  # Replace with the actual URL
# temp=[]
# Send an HTTP request to fetch the page content
    response = requests.get(url)
    # print(ele)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        links = [a["href"] for a in soup.find_all("a", href=True) if ele in a.get_text()]
        curr_data={}
        curr_data['venue']=ele
        try:
            response = requests.get(links[0])

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                # all_points=soup.find_all('ul')
                # print(all_points)
                all_points = soup.find_all('ul')
                

                # Filter the <ul> elements containing the word 'Floodlights'
                # filtered_points = [ul.get_text(strip=True) for ul in all_points if 'Floodlights' in ul.get_text()]
                # filtered_points_2 = [ul.get_text(strip=True) for ul in all_points if 'Leg Side Boundary Length' in ul.get_text()]
                
                # for point in filtered_points:
                #     print(point)
                # for point in filtered_points_2:
                #     print(point)

                # filtered_points = [li for ul in all_points for li in ul.find_all('li') if 'Floodlights' in li.get_text()]
                # filtered_points_2 = [li for ul in all_points for li in ul.find_all('li') if 'Leg Side Boundary Length' in li.get_text()]

                # # Print Floodlights-related list items
                # print("Floodlights-related points:")
                # for li in filtered_points:
                #     print(li.get_text(strip=True))

                # # Print Boundary-related list items
                # print("\nBoundary-related points:")
                # for li in filtered_points_2:
                #     print(li.get_text(strip=True))

                filtered_points = [ul.find_all('li') for ul in all_points if any('Floodlights' in li.get_text() for li in ul.find_all('li'))]
                filtered_points_2 = [ul.find_all('li') for ul in all_points if any('Leg Side Boundary Length' in li.get_text() for li in ul.find_all('li'))]

                # Print all li elements from uls containing 'Floodlights'
                # print("All li elements from uls containing 'Floodlights':")
                # print(filtered_points)
                for li_list in filtered_points:
                    for li in li_list:
                        # print(li)
                        li=li_text=li.get_text(strip=True)
                        if('Field size:' in li):
                            match=re.search(r"Field size:(.+)", li)
                                
                            if match:
                                curr_data['Field size'] = match.group(1)[1:]
                        
                        if('Surface:' in li):
                                
                            match=re.search(r"Field size:(.+)", li)
                                
                            if match:
                                curr_data['Surface'] = match.group(1)[1:]
                                # curr_data['Surface']=re.search(r"Surface:(.+)", li)
                        
                        if('Boundary Length:' in li):
                            if match:
                                curr_data['Boundary Length'] = match.group(1)[1:]
                                # curr_data['Boundary Length']=re.search(r"Boundary Length (Dimension):(.+)", li)
                                
                        # print(li.get_text(strip=True))

                # # Print all li elements from uls containing 'Leg Side Boundary Length'
                # # print("\nAll li elements from uls containing 'Leg Side Boundary Length':")
                for li_list in filtered_points_2:
                
                    for li in li_list:
                        # print(li)
                        # li_text = str(li)  # Make sure it's a string
                        li_text=li.get_text(strip=True)
                        if 'Leg Side Boundary Length:' in li_text:
                            match = re.search(r"Leg Side Boundary Length:(.+)", li_text)
                            if match:
                                curr_data['Leg Side Boundary Length'] = match.group(1)[1:]

                        if 'Off Side Boundary Length:' in li_text:
                            match = re.search(r"Off Side Boundary Length:(.+)", li_text)
                            if match:
                                curr_data['Off Side Boundary Length'] = match.group(1)[1:]

                        if 'Straight Boundary Length:' in li_text:
                            match = re.search(r"Straight Boundary Length:(.+)", li_text)
                            if match:
                                curr_data['Straight Boundary Length'] = match.group(1)[1:]

                        if 'Back Side Boundary Length:' in li_text:
                            match = re.search(r"Back Side Boundary Length:(.+)", li_text)
                            if match:
                                curr_data['Back Side Boundary Length'] = match.group(1)[1:]

                # for li_list in filtered_points_2:
                    # for li in li_list:
                    #     li=str(li)
                    #     if('Leg Side Boundary Length:' in li):
                    #             curr_data['Leg Side Boundary Length']=re.search(r"Leg Side Boundary Length:(.+)", li)
                    #     if('Off Side Boundary Length:' in li):      
                    #             curr_data['Off Side Boundary Length']=re.search(r"Off Side Boundary Length:(.+)", li)
                    #     if('Straight Boundary Length:' in li):
                    #             curr_data['Straight Boundary Length']=re.search(r"Straight Boundary Length:(.+)", li)
                    #     if('Back Side Boundary Length:' in li):
                    #             curr_data['Back Side Boundary Length']=re.search(r"Back Side Boundary Length:(.+)", li)
                    #             {'Leg Side Boundary Length': <re.Match object; span=(4, 34), match='Leg Side Boundary Length:</li>'>, 'Off Side Boundary Length': <re.Match object; span=(4, 34), match='Off Side Boundary Length:</li>'>, 'Straight Boundary Length': <re.Match object; span=(4, 34), match='Straight Boundary Length:</li>'>, 'Back Side Boundary Length': <re.Match object; span=(4, 35), match='Back Side Boundary Length:</li>'>}

                h3_tag = soup.find('h3', string="Who Will Win / Match Winner Prediction")
                # print(h3_tag)
                if h3_tag:
                    # Go 3 divs out and find the next next sibling (which contains the numbers)
                    target_div = h3_tag.find_parent('div').find_parent('div').find_parent('div').find_parent('div').find_next_sibling().find_next_sibling()
                    # print(target_div)
                    # Extract percentages or numbers
                    # numbers = [div.get_text(strip=True) for div in target_div.find_all('div')]
                    # print("Extracted numbers:", numbers)
                    percentage_spans=target_div.find_all('span', class_='elementor-progress-percentage')

                    percentages = [span.get_text(strip=True) for span in percentage_spans]
                    print("Extracted Percentages:", percentages)
                    curr_data['batting_1st']=percentages[0]
                    curr_data['bowling_1st']=percentages[1]


                else:
                    print("Tag not found.")

                divs = soup.find_all("h3")
                div=[div for div in divs if "Batting Pitch Or Bowling Pitch" in div.get_text(strip=True)]

                d=div[0].find_parent("div").find_parent("div").find_parent("div").find_parent("div")
                d=d.find_next_sibling()
                pitch_desc=[p.get_text() for p in d.find_all('p')]
                # print(pitch_desc)
                curr_data['desc']=pitch_desc

            # print(curr_data)
            info_type1_error.append(curr_data)
        except:
            # print({})
            info_type1_error.append(curr_data)
    # pass
    # break
       

Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['57%', '43%']
Extracted Percentages: ['65%', '35%']
Extracted Percentages: ['100%', '100%']
Extracted Percentages: ['30%', '70%']
Extracted Percentages: ['100%', '100%']
Extracted Percentages: ['70%', '30%']
Extracted Percentages: ['29%', '71%']
Extracted Percentages: ['100%', '100%']
Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['70%', '30%']
Extracted Percentages: ['100%', '100%']
Extracted Percentages: ['70%', '30%']
Extracted Percentages: ['40%', '60%']
Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['70%', '30%']
Extracted Percentages: ['55%', '45%']
Extracted Percentages: ['100%', '100%']
Extracted Percentages: ['53%', '47%']
Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['40%', '60%']
Extracted Percentages: ['100%', '100%']
Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['60%', '40%']
Extracted Percentages: ['33%', '67%']


In [131]:
desc_list=[]
for entry in info_type1_error:
    try:
        var=entry['desc']
        if(len(var)>1):
            var=var[0]+" "+var[1]
        else: 
            var=var[0]
        desc_list.append(var)
    except:
        # print('Batting or Bowling Pitch')
        desc_list.append('Batting or Bowling Pitch')

In [145]:
# desc_list
# from claude_ai
pitch_info=['Ba', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Na', 'Ba', 'Na', 'Ba', 'Bo', 'Bo', 'Ba', 'Na', 'Ba', 'Bo', 'Na', 'Na', 'Na', 'Ba', 'Na', 'Ba', 'Na', 'Ba', 'Na', 'Bo', 'Bo', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Ba', 'Ba', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Bo', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Ba', 'Na', 'Na', 'Bo', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Ba', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Na', 'Bo', 'Na', 'Ba', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Ba', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Ba', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Na', 'Na', 'Na', 'Bo', 'Na', 'Na', 'Bo', 'Na', 'Ba', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Ba', 'Na', 'Na', 'Na', 'Ba', 'Bo', 'Na', 'Ba', 'Ba', 'Ba', 'Bo', 'Na', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Bo', 'Bo', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Bo', 'Na', 'Na', 'Na', 'Ba', 'Ba', 'Na', 'Bo', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Na', 'Ba', 'Na', 'Na', 'Na']

In [7]:
import copy
temp=copy.deepcopy(info_type1_error)

In [147]:
len(pitch_info)

168

In [8]:
for i in range(len(temp)):
    temp[i]['pitch']=pitch_info[i]

NameError: name 'pitch_info' is not defined

In [154]:
for i in range(len(temp)):
    temp[i]['match_ids']=venue_matches[temp[i]['venue']]

In [9]:
with open("pitch_data2.json", "w") as json_file:
    json.dump(temp, json_file, indent=4) 

In [84]:
import copy
temp=copy.deepcopy(info_type1_error)

In [45]:
import re

data = """
(Your provided text)
"""

boundary_pattern = r"(\w+ Side Boundary Length):\s*(\d+ meters|Not available)"

sets = re.split(r"Boundary Set \d+:", data)
parsed_boundaries = []

for boundary_set in sets[1:]:
    boundaries = dict(re.findall(boundary_pattern, boundary_set))
    parsed_boundaries.append(boundaries)

# Print the organized output
for i, boundaries in enumerate(parsed_boundaries, 1):
    print(f"Boundary Set {i}:")
    for key in ["Leg Side Boundary Length", "Off Side Boundary Length", "Straight Boundary Length", "Back Side Boundary Length"]:
        value = boundaries.get(key, "Not available")
        print(f"  {key}: {value}")
    print()


In [22]:
import re
import spacy
import json

# Load the spaCy model for NLP
nlp = spacy.load("en_core_web_sm")

# Input data
data = [
    'Field size: 175 mtr x 135 mtr',
    'Surface: Grass',
    'Boundary Length (Dimension): 191 Yd x 147 Yd',
    'Leg Side Boundary Length: 63 meters',
    'Off Side Boundary Length: 72 meters',
    'Straight Boundary Length: 96 meters',
    'Back Side Boundary Length: 79 meters',
    'Bellerive Oval Cricket Ground is a Batting Pitch.',
    'Bellerive Oval Cricket Ground is a Batting Pitch. Pitches typically start off slow and gradually flatten out as the match progresses. Over time, the pitch characteristics have evolved, leading to a more balanced contest between bat and ball. While bowlers may enjoy some initial assistance, particularly early on, the conditions tend to favor the batsmen as the game moves forward, making for an exciting and competitive encounter.',
    'Hobart'
]

# Regex patterns to extract relevant data
patterns = {
    'field_size': r"Field size:\s*(\d+\s*mtr\s*x\s*\d+\s*mtr)",
    'surface': r"Surface:\s*(\w+)",
    'boundary_length': r"Boundary Length \(Dimension\):\s*(\d+\s*Yd\s*x\s*\d+\s*Yd)",
    'leg_side_boundary': r"Leg Side Boundary Length:\s*(\d+\s*meters)",
    'off_side_boundary': r"Off Side Boundary Length:\s*(\d+\s*meters)",
    'straight_boundary': r"Straight Boundary Length:\s*(\d+\s*meters)",
    'back_side_boundary': r"Back Side Boundary Length:\s*(\d+\s*meters)",
    'pitch_type': r"Cricket Ground is a (Batting Pitch|Bowling Pitch)",
    'location': r"(\w+)$"
}

# Initialize a dictionary to store the extracted values
extracted_data = {}
data_pitch_info=[]
# Extract values using regex
for key, pattern in patterns.items():
    match = re.search(pattern, ' '.join(data))
    curr_match=(match.group(1))
    data_pitch_info.append(curr_match)
    if match:
        extracted_data[key] = match.group(1)

# For pitch description, you can analyze the text further if needed
pitch_description = [entry for entry in data if "Pitch" in entry]
if pitch_description:
    extracted_data['pitch_description'] = ' '.join(pitch_description)

# # Output the result in JSON format
# json_result = json.dumps(extracted_data, indent=4)
print(data_pitch_info)


['175 mtr x 135 mtr', 'Grass', '191 Yd x 147 Yd', '63 meters', '72 meters', '96 meters', '79 meters', 'Batting Pitch', 'Hobart']


In [ ]:

# file_path = "output2.csv"
# try:
#     with open(file_path, mode='w', newline='') as file:
#         writer = csv.writer(file)
        
#         # Write the rows of the matrix into the CSV
#         writer.writerows(final_data)
        
#     print(f"Matrix successfully written to {file_path}")
    
# except Exception as e:
#     print(f"An error occurred: {e}")

In [ ]:
final_data